In [40]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader

from sklearn.model_selection import train_test_split

In [41]:
torch.manual_seed(0)
np.random.seed(0)

In [42]:
df = pd.read_csv("/Users/michaelhein/Programs/Dicebot/outputs/output.txt")

In [43]:
df.head()

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
0,25,Agg-High,5,9,4,2,0,0,3,0,0
1,24,Agg-Low,5,8,4,0,0,1,2,2,0
2,24,Cond-Low,5,9,2,1,2,1,0,0,1
3,23,Cond-High,5,9,3,1,0,3,0,1,0
4,22,Conserv,4,1,4,0,1,0,2,0,1


In [44]:
df["totalDice"]

0         25
1         24
2         24
3         23
4         22
          ..
590880     6
590881     6
590882     5
590883     4
590884     4
Name: totalDice, Length: 590885, dtype: int64

In [45]:
df.shape

(590885, 11)

In [46]:
df[["totalDice", "player", "numPlayerDice", "quantityDeclared", "diceNumDeclared"]]

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared
0,25,Agg-High,5,9,4
1,24,Agg-Low,5,8,4
2,24,Cond-Low,5,9,2
3,23,Cond-High,5,9,3
4,22,Conserv,4,1,4
...,...,...,...,...,...
590880,6,Agg-High,1,2,5
590881,6,Agg-Low,1,3,4
590882,5,Cond-High,3,2,3
590883,4,Agg-Low,1,1,4


In [47]:
features = df[df.columns[:5]]
features.head()

,totalDice,player,numPlayerDice,quantityDeclared,diceNumDeclared
0,25,Agg-High,5,9,4
1,24,Agg-Low,5,8,4
2,24,Cond-Low,5,9,2
3,23,Cond-High,5,9,3
4,22,Conserv,4,1,4


In [48]:
targets = df[df.columns[5:]]
targets.head()

,1s,2s,3s,4s,5s,6s
0,2,0,0,3,0,0
1,0,0,1,2,2,0
2,1,2,1,0,0,1
3,1,0,3,0,1,0
4,0,1,0,2,0,1


In [49]:
AggLow = df[df["player"] == "Agg-Low"]
AggLow = AggLow.drop(columns = ["player"])
AggLow.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
1,24,5,8,4,0,0,1,2,2,0
6,22,5,7,4,2,0,0,2,0,1
11,21,5,7,4,1,0,1,2,1,0
16,19,5,6,3,2,0,2,0,0,1
21,18,5,6,4,1,1,0,1,0,2


In [50]:
td, test = train_test_split(AggLow, test_size=0.1)

In [51]:
train, develop = train_test_split(AggLow, test_size=0.2)

In [52]:
train.shape

(101144, 10)

In [53]:
develop.shape

(25286, 10)

In [54]:
test.shape

(12643, 10)

In [55]:
x_train = train[train.columns[:4]]
y_train = train[train.columns[4:]]

x_develop = develop[develop.columns[:4]]
y_develop = develop[develop.columns[4:]]

x_test = test[test.columns[:4]]
y_test = test[test.columns[4:]]

In [56]:
x_train_tensor = torch.from_numpy(x_train.to_numpy()).float()
y_train_tensor = torch.from_numpy(y_train.to_numpy()).float()

x_develop_tensor = torch.from_numpy(x_develop.to_numpy()).float()
y_develop_tensor = torch.from_numpy(y_develop.to_numpy()).float()

x_test_tensor = torch.from_numpy(x_test.to_numpy()).float()
y_test_tensor = torch.from_numpy(y_test.to_numpy()).float()

In [57]:
train_dataset = list(zip(x_train_tensor, y_train_tensor))
develop_dataset = list(zip(x_develop_tensor, y_develop_tensor))
test_dataset = list(zip(x_test_tensor, y_test_tensor))

In [58]:
class Net(nn.Module):
    def __init__(self, num_layers, layers_size):
        super(Net, self).__init__()
        self.hidden_layers = nn.ModuleList([nn.Linear(4, layers_size)])
        self.hidden_layers.extend([nn.Linear(layers_size, layers_size) for i in range(1, num_layers-1)])
        self.output = nn.Linear(layers_size, 6)
        self.activation = nn.ReLU()
        
    def forward(self, x):
        z = x
        for layer in self.hidden_layers:
            z = self.activation(layer(z))
        return self.output(z)

In [59]:
class Trainer():
    def __init__(self, net, optim, loss_function, train_loader):
        self.net = net
        self.optim = optim
        self.loss_function = loss_function
        self.train_loader = train_loader
        
    def train(self, epochs, verbose=True):
        losses = []
        for epoch in range(epochs):
            epoch_loss = 0.0
            epoch_steps = 0
            for data in self.train_loader:
                x = data[0]
                y = data[1]
                
                self.optim.zero_grad()
                
                preds = self.net.forward(x)
                
                loss = self.loss_function(preds, y)
                
                loss.backward()
                
                self.optim.step()
                
                epoch_loss += loss.item()
                epoch_steps += 1
                
            losses.append(epoch_loss / epoch_steps)
            if verbose: print("epoch [%d]: loss %.3f" % (epoch+1, losses[-1]))
        return losses

In [60]:
## Hyperparamters

num_epochs = 30
batch_size = 256
learning_rate = 0.02

In [61]:
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, num_workers=0)
develop_loader = DataLoader(dataset=develop_dataset, batch_size=batch_size, shuffle=False, num_workers=0)

In [62]:
net = Net(2, 16)
opt = optim.Adam(net.parameters(), lr=learning_rate)
loss_function = nn.MSELoss()

trainer = Trainer(net=net, optim=opt, loss_function=loss_function, train_loader=train_loader)

losses = trainer.train(num_epochs)

epoch [1]: loss 0.410
epoch [2]: loss 0.354
epoch [3]: loss 0.330
epoch [4]: loss 0.317
epoch [5]: loss 0.291
epoch [6]: loss 0.286
epoch [7]: loss 0.282
epoch [8]: loss 0.281
epoch [9]: loss 0.280
epoch [10]: loss 0.279
epoch [11]: loss 0.278
epoch [12]: loss 0.278
epoch [13]: loss 0.277
epoch [14]: loss 0.275
epoch [15]: loss 0.276
epoch [16]: loss 0.275
epoch [17]: loss 0.274
epoch [18]: loss 0.273
epoch [19]: loss 0.273
epoch [20]: loss 0.272
epoch [21]: loss 0.275
epoch [22]: loss 0.271
epoch [23]: loss 0.273
epoch [24]: loss 0.272
epoch [25]: loss 0.273
epoch [26]: loss 0.273
epoch [27]: loss 0.270
epoch [28]: loss 0.270
epoch [29]: loss 0.272
epoch [30]: loss 0.271


In [63]:
AggLow.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared,1s,2s,3s,4s,5s,6s
1,24,5,8,4,0,0,1,2,2,0
6,22,5,7,4,2,0,0,2,0,1
11,21,5,7,4,1,0,1,2,1,0
16,19,5,6,3,2,0,2,0,0,1
21,18,5,6,4,1,1,0,1,0,2


In [64]:
train_dataset[0]

(tensor([13.,  2.,  4.,  1.]), tensor([1., 0., 1., 0., 0., 0.]))

In [65]:
train_dataset[1]

(tensor([8., 2., 2., 4.]), tensor([0., 1., 0., 1., 0., 0.]))

In [66]:
x_train.head()

,totalDice,numPlayerDice,quantityDeclared,diceNumDeclared
577794,13,2,4,1
526856,8,2,2,4
406465,25,5,8,5
63683,14,3,4,4
116196,14,3,4,5


In [67]:
net.forward(train_dataset[0][0])

tensor([ 1.0615,  0.3276, -0.1647,  0.1028,  0.1926,  0.4106],
       grad_fn=<AddBackward0>)

In [68]:
net.forward(train_dataset[1][0])

tensor([0.1513, 0.1299, 0.0621, 1.3643, 0.1004, 0.1923],
       grad_fn=<AddBackward0>)

In [69]:
develop_dataset[0]

(tensor([15.,  5.,  5.,  1.]), tensor([2., 0., 1., 1., 1., 0.]))

In [70]:
develop_dataset[1]

(tensor([24.,  4.,  8.,  2.]), tensor([1., 2., 0., 0., 0., 1.]))

In [71]:
develop_dataset[2]

(tensor([4., 2., 2., 2.]), tensor([0., 1., 0., 0., 0., 1.]))

In [72]:
develop_dataset[3]

(tensor([8., 3., 3., 1.]), tensor([2., 0., 0., 1., 0., 0.]))

In [73]:
net.forward(develop_dataset[0][0])

tensor([1.6910, 0.7712, 0.3061, 0.4807, 0.6410, 0.9987],
       grad_fn=<AddBackward0>)

In [74]:
net.forward(develop_dataset[1][0])

tensor([0.3524, 1.7205, 0.2619, 0.3981, 0.4535, 0.7430],
       grad_fn=<AddBackward0>)

In [75]:
net.forward(develop_dataset[2][0])

tensor([0.0312, 1.1980, 0.0804, 0.1427, 0.1657, 0.3507],
       grad_fn=<AddBackward0>)

In [76]:
net.forward(develop_dataset[3][0])

tensor([1.2931, 0.4227, 0.0245, 0.2262, 0.3480, 0.6051],
       grad_fn=<AddBackward0>)

In [77]:
torch.save(net.state_dict(), "/Users/michaelhein/Programs/Dicebot/models/aggressive_low_lvl_stat.obj")